In [21]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas_ta.volatility import atr
from pandas_ta.trend import adx
import matplotlib.pyplot as plt

import plotly.graph_objects as go

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import resample, zigzag, supertrend, get_pivots_alert, technical_summary
from scripts.functions import ohlc_mirror_v


C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [22]:
klines_files = []
klines_files.append('./backtest/klines/0m01/Alts_GMXUSDT_0m01_2024-05-28_2025-05-28.DataFrame')
klines_files.append('./backtest/klines/0m01/Alts_AAVEUSDT_0m01_2024-05-28_2025-05-28.DataFrame')
klines_files.append('./backtest/klines/0m01/Alts_GALAUSDT_0m01_2024-05-28_2025-05-28.DataFrame')
klines_files.append('./backtest/klines/0m01/Alts_AVAXUSDT_0m01_2024-05-28_2025-05-28.DataFrame')

#klines_files.append('./backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame')
#klines_files.append('./backtest/klines/0m01/Completo_BNBUSDT_0m01_2021-01-01_2023-07-31.DataFrame')
#klines_files.append('./backtest/klines/0m01/Completo_ETHUSDT_0m01_2021-01-01_2023-07-31.DataFrame')



In [23]:
ALERT_THRESHOLD = 1.5

def contar_decimales(f):
    s = str(f)
    if '.' in s:
        return len(s.split('.')[1].rstrip('0'))
    return 0

def get_alert(df):
    data = {}
    data['alert'] = 0
    df_last100 = df[-100:]
    last100_high = df_last100['high'].max()
    last100_min = df_last100['low'].min()
    variacion_pct = (last100_high - last100_min) / last100_min * 100
    if abs(variacion_pct) >= (2*ALERT_THRESHOLD):

        min_periods = 50
        price = df.iloc[-1]['close']
        if df['close'].count() >= min_periods:
            
            adx_length = 14
            adx_data = adx(df['high'],df['low'],df['close'],length=adx_length)
            last_adx = adx_data.iloc[-1][f'ADX_{adx_length}']
            adx_data['adx_ma'] = adx_data[f'ADX_{adx_length}'].rolling(window=5).mean()
            last_adx_ma = adx_data.iloc[-1]['adx_ma']
                    
            if last_adx > 26:
                
                df = supertrend(df)
                trend = int(df.iloc[-1]['st_trend'])
                
                if trend >= 1 or trend <= -1:

                    df = zigzag(df)
                    pivots = df[df['ZigZag']>0]['ZigZag'].tolist()
                    
                    if len(pivots) >= 5: #Se busca que existan mas pivots de lo necesario par apoder formarlos

                        if trend >=1:
                            #Alertas en LONG
    
                            is_uptrend_structure = pivots[-1] > pivots[-3] and pivots[-2] > pivots[-4]
                            last_high_pivot = pivots[-1]
                            if is_uptrend_structure and price > last_high_pivot:
                                data['alert_str'] = 'Breakout LONG'
                                data['alert'] = 1
                                data['side'] = 1
                                data['sl1'] = pivots[-2] # O min_flp si es más conservador
                                data['tp1'] = last_high_pivot + (last_high_pivot - pivots[-2]) # Proyección 1:1 del último impulso
                                data['in_price'] = last_high_pivot 

                            # Necesitas al menos 5 pivots para esto
                            high_pivot_1 = pivots[-1]
                            low_pivot_1 = pivots[-2]
                            high_pivot_2 = pivots[-3]
                            low_pivot_2 = pivots[-4]
                            # Chequea si los máximos están en un nivel similar (con una tolerancia)
                            resistance_level_formed = abs(high_pivot_1 - high_pivot_2) < (high_pivot_1 * 0.005) # ej: 0.5% de tolerancia
                            # Chequea si los mínimos son ascendentes
                            higher_lows_formed = low_pivot_1 > low_pivot_2
                            if resistance_level_formed and higher_lows_formed and price > high_pivot_1:
                                # ¡ALERTA DE TRIÁNGULO ASCENDENTE LONG!
                                data['alert_str'] = 'Ascending Triangle Breakout'
                                data['alert'] = 2
                                data['side'] = 1
                                data['sl1'] = low_pivot_1 
                                data['tp1'] = high_pivot_1 + (high_pivot_1-low_pivot_1)
                                data['in_price'] = high_pivot_1
                                
                            # Viniendo de una tendencia bajista (ej: Supertrend era negativo)
                            low_pivot_1 = pivots[-2]
                            neckline_pivot = pivots[-3]
                            low_pivot_2 = pivots[-4]
                            # Chequea si los mínimos están al mismo nivel (tolerancia)
                            support_level_formed = abs(low_pivot_1 - low_pivot_2) < (low_pivot_1 * 0.005)
                            # Chequea que el neckline esté por encima de los mínimos
                            is_valid_pattern = neckline_pivot > low_pivot_1
                            if support_level_formed and is_valid_pattern and price > neckline_pivot:
                                data['alert_str'] = 'Double Bottom Breakout'
                                data['alert'] = 3
                                data['side'] = 1
                                data['in_price'] = neckline_pivot
                                data['sl1'] = low_pivot_1 # O el promedio de los dos mínimos
                                data['tp1'] = neckline_pivot + (neckline_pivot - low_pivot_1) # Proyección de la altura del patrón

                            
                        elif trend <= -1:
                            # --- Alertas en SHORT ---

                            # --- 1. Patrón: Breakout SHORT (Contraparte del Breakout LONG) ---
                            # Lógica: Buscamos una estructura de máximos y mínimos descendentes (tendencia bajista)
                            # y entramos cuando el precio rompe el último mínimo, esperando continuación.
                            is_downtrend_structure = pivots[-1] < pivots[-3] and pivots[-2] < pivots[-4]
                            last_low_pivot = pivots[-2]

                            # La alerta se dispara cuando el precio rompe (cae por debajo de) el último mínimo
                            if is_downtrend_structure and price < last_low_pivot:
                                data['alert_str'] = 'Breakout SHORT'
                                data['alert'] = -1 # Usa un ID de alerta diferente para shorts
                                data['side'] = -1 # -1 para SHORT
                                data['in_price'] = last_low_pivot
                                data['sl1'] = pivots[-1] # El Stop Loss va por encima del último máximo
                                # Proyección 1:1 del último impulso bajista
                                data['tp1'] = last_low_pivot - (pivots[-1] - last_low_pivot)
                                # Aquí podrías añadir tu lógica de envío de alerta y salir del bucle si es necesario

                            # --- 2. Patrón: Descending Triangle Breakout (Contraparte del Ascending Triangle) ---
                            # Lógica: Un nivel de soporte plano (mínimos al mismo nivel) y máximos descendentes
                            # que presionan el soporte hasta que se rompe.
                            
                            # Aseguramos que tenemos suficientes pivots para el patrón
                        
                            high_pivot_1 = pivots[-1]
                            low_pivot_1 = pivots[-2]
                            high_pivot_2 = pivots[-3]
                            low_pivot_2 = pivots[-4]

                            # Chequea si los mínimos están en un nivel similar (soporte plano)
                            support_level_formed = abs(low_pivot_1 - low_pivot_2) < (low_pivot_1 * 0.005) # ej: 0.5% de tolerancia

                            # Chequea si los máximos son descendentes
                            lower_highs_formed = high_pivot_1 < high_pivot_2

                            # La alerta se dispara cuando el precio rompe el soporte
                            if support_level_formed and lower_highs_formed and price < low_pivot_1:
                                data['alert_str'] = 'Descending Triangle Breakout'
                                data['alert'] = -2 # Usa un ID de alerta diferente
                                data['side'] = -1
                                data['in_price'] = low_pivot_1
                                data['sl1'] = high_pivot_1 # El SL va por encima del último máximo del triángulo
                                # Proyecta la altura del triángulo hacia abajo desde el punto de ruptura
                                data['tp1'] = low_pivot_1 - (high_pivot_1 - low_pivot_1)
                                # Lógica de envío de alerta

                            # --- 3. Patrón: Double Top Breakout (Contraparte del Double Bottom) ---
                            # Lógica: Patrón de reversión. El precio intenta superar una resistencia dos veces y falla,
                            # luego rompe el soporte intermedio ("neckline"), señalando un cambio a tendencia bajista.

                            # Aseguramos que tenemos suficientes pivots para el patrón
                            high_pivot_1 = pivots[-1]
                            neckline_pivot_short = pivots[-2] # El "neckline" en un doble techo es el mínimo intermedio
                            high_pivot_2 = pivots[-3]

                            # Chequea si los máximos están al mismo nivel (resistencia)
                            resistance_level_formed = abs(high_pivot_1 - high_pivot_2) < (high_pivot_1 * 0.005)

                            # Chequea que el neckline esté por debajo de los máximos
                            is_valid_pattern = neckline_pivot_short < high_pivot_1

                            # La alerta se dispara cuando el precio rompe el neckline hacia abajo
                            if resistance_level_formed and is_valid_pattern and price < neckline_pivot_short:
                                data['alert_str'] = 'Double Top Breakout'
                                data['alert'] = -3 # Usa un ID de alerta diferente
                                data['side'] = -1
                                data['in_price'] = neckline_pivot_short
                                data['sl1'] = high_pivot_1 # SL por encima del doble techo
                                # Proyecta la altura del patrón hacia abajo desde el neckline
                                data['tp1'] = neckline_pivot_short - (high_pivot_1 - neckline_pivot_short)
                                # Lógica de envío de alerta                        

            #Data general de las alertas
            if data['alert'] != 0:
                decs = max(contar_decimales(data['sl1']), contar_decimales(data['tp1']))
                data['in_price'] = round(data['in_price'],decs)
                data['df'] = df
                data['datetime'] = df.iloc[-1]['datetime']
                data['pivots'] = pivots
                data['adx'] = last_adx
                data['adx_ma'] = last_adx_ma

    return data

In [24]:
print('Inicia el proceso')
minutes_x_kline = 15
# --- Parámetros ---
MAX_BIGMIN_CANDLES = 200 # Número máximo de velas de BIGmin a mantener

trade_columns = ['type','side','result','open','buy','sell','close','tp','sl','adx','adx_ma','in_price']
trades = pd.DataFrame(columns=trade_columns)

tot_klines_files = len(klines_files)
act_klines_file = 0
for klines_file in klines_files:
    # --- Inicialización ---
    # DataFrame para las velas de BIG minutos que se van formando
    dfBIG_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']
    dfBIG = pd.DataFrame(columns=dfBIG_cols)

    act_klines_file += 1
    print(f'\n{klines_file}')
    with open(klines_file, 'rb') as file:
        df = pickle.load(file)
    #df = df[50000:150000]
    #df.reset_index(inplace=True, drop=True)
    #df = ohlc_mirror_v(df)

    last_BIGmin_candle_start_time = None
    alert = {}

    trade = {}
    tp = None
    sl = None
    total_velas = len(df)
    progreso = 0
    kline_minutes = 0
    status = 0 # (0: Find Alerts, 1: Open Order, 2: Open Pos)
    # --- Iteración sobre el DataFrame de 1 minuto ---
    for row_1min in df.itertuples(index=False): # index=False para no incluir el índice de df_1min
        progreso+=1
        progreso_perc = (progreso/total_velas)*100
        tot_trades = len(trades)
        
        print(f'Procesando {act_klines_file}/{tot_klines_files}: {progreso_perc:.2f}%    {tot_trades}  ',end="\r")
        current_1min_timestamp = row_1min.datetime # Accedemos a la columna datetime
        current_1min_data = row_1min # El namedtuple entero

        this_BIGmin_slot_start_time = current_1min_timestamp.floor(f'{minutes_x_kline}T')

        is_new_BIGmin_candle = False
        # Comparamos con la columna 'datetime' de la última fila de dfBIG
        if len(dfBIG)==0 or this_BIGmin_slot_start_time > dfBIG['datetime'].iloc[-1]:
            is_new_BIGmin_candle = True

        if is_new_BIGmin_candle:
            new_BIGmin_candle_data = {
                'datetime': this_BIGmin_slot_start_time,
                'open': current_1min_data.open, # Acceso a campos del namedtuple
                'high': current_1min_data.high,
                'low': current_1min_data.low,
                'close': current_1min_data.close,
                'volume': current_1min_data.volume
            }
            
            # Añadir como una nueva fila. pd.concat es más robusto aquí.
            new_row_df = pd.DataFrame([new_BIGmin_candle_data])
            dfBIG = pd.concat([dfBIG, new_row_df], ignore_index=True) # ignore_index para mantener el índice numérico secuencial

            if len(dfBIG) > MAX_BIGMIN_CANDLES:
                dfBIG = dfBIG.iloc[-MAX_BIGMIN_CANDLES:].reset_index(drop=True) # reset_index para mantener secuencia 0..N-1
            kline_minutes = 1
        else:
            # Actualizar la última fila de dfBIG usando su índice numérico (.iloc[-1] o len(dfBIG)-1)
            last_row_index = len(dfBIG) - 1
            dfBIG.loc[last_row_index, 'high'] = max(dfBIG.loc[last_row_index, 'high'], current_1min_data.high)
            dfBIG.loc[last_row_index, 'low'] = min(dfBIG.loc[last_row_index, 'low'], current_1min_data.low)
            dfBIG.loc[last_row_index, 'close'] = current_1min_data.close
            dfBIG.loc[last_row_index, 'volume'] += current_1min_data.volume
            # 'datetime' y 'open' no cambian
            kline_minutes +=1

        SR_THRESHOLD = 0.1
        SR_THRESHOLD_STRONG = 0.5
        if len(dfBIG) >= MAX_BIGMIN_CANDLES:

            if status==0:
                if kline_minutes==minutes_x_kline:
                    alert = get_pivots_alert(dfBIG[-150:],threshold=1.5)
                    #brief = technical_summary(dfBIG[-150:])
                    #sr = brief['sentiment_ratio']

                    if alert['alert']!=0:
                        if True:
                        #if (alert['side'] > 0 and sr>SR_THRESHOLD and sr<SR_THRESHOLD_STRONG) or \
                        #   (alert['side'] < 0 and sr<-SR_THRESHOLD and sr>-SR_THRESHOLD_STRONG) :
                            status = 1
                            trade = {'open': current_1min_timestamp,
                                    'close': None,
                                    'side': alert['side'],
                                    'buy': None,
                                    'sell': None,
                                    'sl': alert['sl1'],
                                    'tp': alert['tp1'],
                                    'adx': alert['adx'],
                                    'adx_ma': alert['adx_ma'],
                                    'type': alert['alert_str'],
                                    'in_price': alert['in_price'],
                                    }
                            tp = alert['tp1']
                            sl = alert['sl1']
                    
            elif status == 1: #Open Order
                price = row_1min.close
                high = row_1min.high
                low = row_1min.low
                if low <= trade['in_price'] <= high:
                    trade['buy'] = price if trade['side'] > 0 else None
                    trade['sell'] = price if trade['side'] < 0 else None
                    status = 2
                elif trade['side']>0 and (low<trade['sl'] or high>trade['tp']):
                    status=0
                elif trade['side']<0 and (high>trade['sl'] or low<trade['tp']):
                    status=0

            elif status == 2: #Open Pos
                price = row_1min.close
                high = row_1min.high
                low = row_1min.low

                if trade['side']>0: #LONG
                    if high >= tp:
                        trade['sell'] = tp
                        trade['close'] = current_1min_timestamp
                        status = 0
                        trades.loc[len(trades)] = trade
                        trade = {}
                    elif low <= sl:
                        trade['sell'] = sl
                        trade['close'] = current_1min_timestamp
                        status = 0
                        trades.loc[len(trades)] = trade
                        trade = {}                    
    
                else: #SHORT
                    if low <= tp:
                        trade['buy'] = tp
                        trade['close'] = current_1min_timestamp
                        status = 0
                        trades.loc[len(trades)] = trade
                        trade = {}
                    if high >= sl:
                        trade['buy'] = sl
                        trade['close'] = current_1min_timestamp
                        status = 0
                        trades.loc[len(trades)] = trade
                        trade = {}
        
        




Inicia el proceso

./backtest/klines/0m01/Alts_GMXUSDT_0m01_2024-05-28_2025-05-28.DataFrame
Procesando 1/4: 100.00%    360  
./backtest/klines/0m01/Alts_AAVEUSDT_0m01_2024-05-28_2025-05-28.DataFrame
Procesando 2/4: 100.00%    721  
./backtest/klines/0m01/Alts_GALAUSDT_0m01_2024-05-28_2025-05-28.DataFrame
Procesando 3/4: 100.00%    1085  
./backtest/klines/0m01/Alts_AVAXUSDT_0m01_2024-05-28_2025-05-28.DataFrame


In [25]:
trades['result'] = ((trades['sell']/trades['buy'] - 1 ) *100).round(4)
trades['adx_force'] = trades['adx']/trades['adx_ma']-1
tot_trades = trades['result'].count()
trades_win = trades[trades['result']> 0 ]['result'].count()
result = trades['result'].sum()

print('Trades:',tot_trades)
print(' Win',round((trades_win/tot_trades)*100,2),'%')
print(' Resultado',round(result,2),'%')

#print('Trades Positivos')
#print(trades[trades['result']>0][['adx','adx_force']].describe())
#print('Trades Negativos')
#print(trades[trades['result']<0][['adx','adx_force']].describe())

resumen = trades.groupby('type').agg(
    cantidad=('result', 'count'),
    resultado_total=('result', 'sum'),
    resultado_mean=('result', 'mean'),
).reset_index()
print(resumen)

Trades: 1397
 Win 48.25 %
 Resultado -99.91 %
                           type  cantidad  resultado_total  resultado_mean
0                 Breakout LONG        13         -23.8164       -1.832031
1                Breakout SHORT       741        -105.3863       -0.142222
2  Descending Triangle Breakout       118         -32.3593       -0.274231
3        Double Bottom Breakout       165         -13.4083       -0.081262
4           Double Top Breakout       360          75.0581        0.208495


In [26]:
"""
type  cantidad  resultado_total  resultado_mean  
0                 Breakout LONG         1          -1.4925       -1.492500   
1                Breakout SHORT       213          21.0852        0.098992   
2  Descending Triangle Breakout        17          25.2164        1.483318   
3        Double Bottom Breakout        24           0.4954        0.020642   
4           Double Top Breakout        52           7.3623        0.141583 
"""

'\ntype  cantidad  resultado_total  resultado_mean  \n0                 Breakout LONG         1          -1.4925       -1.492500   \n1                Breakout SHORT       213          21.0852        0.098992   \n2  Descending Triangle Breakout        17          25.2164        1.483318   \n3        Double Bottom Breakout        24           0.4954        0.020642   \n4           Double Top Breakout        52           7.3623        0.141583 \n'

In [ ]:
import matplotlib.pyplot as plt
trades = trades.sort_values(by='open')

# Calcular el PnL acumulado
trades['pnl'] = trades['result'].cumsum()

# Graficar el PnL
# Crear la figura y el eje
fig, ax = plt.subplots(figsize=(12, 6))

# Graficar PnL acumulado
# Cambia de color según sea positivo o negativo
pnl = trades['cumulative_pnl']
color = ['green' if val >= 0 else 'red' for val in pnl]

ax.plot(trades['open'], pnl, color='black', linewidth=2, label='PnL acumulado')
ax.fill_between(trades['open'], pnl, where=(pnl >= 0), interpolate=True, color='green', alpha=0.3)
ax.fill_between(trades['open'], pnl, where=(pnl < 0), interpolate=True, color='red', alpha=0.3)

# Etiquetas y estilo
ax.set_title('Evolución del PnL acumulado')
ax.set_xlabel('Fecha de apertura')
ax.set_ylabel('PnL acumulado')
ax.grid(True)
ax.legend()

plt.tight_layout()
plt.show()